# PA4: Implementing linear classifiers

Participants: Arvid Nyberg and Alfred Karlsson

## Exercise question

The second training set can be visualized as:

              July   December  
       Sydney rain   sun  
       Paris  sun    rain

There is no way to draw a straight line that separates the two categories, which means they are linearly inseparable.
Since both the Perceptron and LinearSVC are linear classifiers they fail to "memorize" the training set.

In [2]:
import random
import numpy as np
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator

In [3]:


class LinearClassifier(BaseEstimator):
    """
    General class for binary linear classifiers. Implements the predict
    function, which is the same for all binary linear classifiers. There are
    also two utility functions.
    """

    def decision_function(self, X):
        """
        Computes the decision function for the inputs X. The inputs are assumed to be
        stored in a matrix, where each row contains the features for one
        instance.
        """
        return X.dot(self.w)

    def predict(self, X):
        """
        Predicts the outputs for the inputs X. The inputs are assumed to be
        stored in a matrix, where each row contains the features for one
        instance.
        """

        # First compute the output scores
        scores = self.decision_function(X)

        # Select the positive or negative class label, depending on whether
        # the score was positive or negative.
        out = np.select([scores >= 0.0, scores < 0.0],
                        [self.positive_class,
                         self.negative_class])
        return out

    def find_classes(self, Y):
        """
        Finds the set of output classes in the output part Y of the training set.
        If there are exactly two classes, one of them is associated to positive
        classifier scores, the other one to negative scores. If the number of
        classes is not 2, an error is raised.
        """
        classes = sorted(set(Y))
        if len(classes) != 2:
            raise Exception("this does not seem to be a 2-class problem")
        self.positive_class = classes[1]
        self.negative_class = classes[0]

    def encode_outputs(self, Y):
        """
        A helper function that converts all outputs to +1 or -1.
        """
        return np.array([1 if y == self.positive_class else -1 for y in Y])

## Implementing the SVC

We made a copy of the Perceptron algorithm and modified it to implement the Pegasos algorithm. The modifications made to the perception class is changing the algorithm according to the pseudocode given in a4_clarification.pdf. Furthermore the variable lambda was added and given the value 1/N where N is the number of instances of the training data. 

In [4]:
class Pegasos(LinearClassifier):
    """
    A straightforward implementation of the Pegasos learning algorithm.
    """

    def __init__(self, n_iter=20):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the Pegasos learning algorithm.
        """
        
        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
        n_features = X.shape[1]
        self.w = np.zeros(n_features)
        #Set lambda to 1/N where N is number of instances in the training set
        n_instances = X.shape[0]
        lmbda = 1/n_instances

        # Pegasos algorithm:
        t = 0
        for i in range(self.n_iter):

            # Select a random training instance.
            index = random.randint(0, len(X)-1)
            x, y = X[index], Ye[index]
            #Setting the step length
            t += 1
            n = 1/(lmbda*t)

            # Compute the output score for this instance.
            score = x.dot(self.w)

            # Stochastic gradient descent with hinge loss
            if y*score < 1:
                self.w = (1-n*lmbda)*self.w + n*y*x
            else:
                self.w = (1-n*lmbda)*self.w

## Logistic regression

The difference between the Pegasos algorithm and this implementation of LR is just the loss funcion.
The gradient of the log loss is the same as the gradient of the hinge loss woth an added denomiator.
The log loss is also continuous and non-zero for score = 1 only one case is needed.  
  
To implement the LR we have added the demoninator of the gradient of the log loss and removed the if-clause of the stochastic gradient descent.
Otherwise, the implementation is the same as Pegasos.

In [5]:
class LogisticRegression(LinearClassifier):
    """
    A straightforward implementation of the LogisticRegression algorithm.
    """

    def __init__(self, n_iter=20):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the LogisticRegression algorithm.
        """
        
        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
        n_features = X.shape[1]
        self.w = np.zeros(n_features)
        #Set lambda to 1/N where N is number of instances in the training set
        n_instances = X.shape[0]
        lmbda = 1/n_instances

        # Logistic Regression algorithm:
        t = 0
        for i in range(self.n_iter):

            # Select a random training instance.
            index = random.randint(0, len(X)-1)
            x, y = X[index], Ye[index]
            #Setting the step length
            t += 1
            n = 1/(lmbda*t)

            # Compute the output score for this instance.
            score = x.dot(self.w)

            # Stochastic gradient descent with log loss
            self.w = (1-n*lmbda)*self.w + n*x*y/(1+np.exp(y*score))

In [7]:
def read_data(corpus_file):
    X = []
    Y = []
    with open(corpus_file, encoding='utf-8') as f:
        for line in f:
            _, y, _, x = line.split(maxsplit=3)
            X.append(x.strip())
            Y.append(y)
    return X, Y



# Read all the documents.
X, Y = read_data('pa4/data/all_sentiment_shuffled.txt')

# Split into training and test parts.
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2,
                                                random_state=0)

In [12]:
# Train and test the Pegasos algorithm and Logistic Regression for different values of n_iter
for n_iter in [100000, 200000, 500000]:
    print('n_iter =', n_iter)
    for classifier in [Pegasos(n_iter=n_iter), LogisticRegression(n_iter=n_iter)]:
        pipeline = make_pipeline(
            TfidfVectorizer(),
            SelectKBest(k=1000),
            Normalizer(),
            classifier
        )
        t0 = time.time()
        pipeline.fit(Xtrain, Ytrain)
        t1 = time.time()
        print('Training time: {:.2f} sec.'.format(t1-t0))
        Yguess = pipeline.predict(Xtest)
        print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))
    print()

n_iter = 100000
Training time: 4.31 sec.
Accuracy: 0.8359.
Training time: 5.48 sec.
Accuracy: 0.8326.

n_iter = 200000
Training time: 5.35 sec.
Accuracy: 0.8326.
Training time: 8.49 sec.
Accuracy: 0.8347.

n_iter = 500000
Training time: 10.78 sec.
Accuracy: 0.8347.


C:\Users\arvid\AppData\Local\Temp\ipykernel_24680\2947770861.py:52: RuntimeWarning: overflow encountered in exp
  self.w = (1-n*lmbda)*self.w + n*x*y/(1+np.exp(y*score))


Training time: 18.24 sec.
Accuracy: 0.8376.



For these values of n_iter the model perform about the same, while the Logistic Regression takes longer time because of its computational complexity.